In [1]:
import tensorflow as tf
import sys

import numpy as np

import labelreg.helpers as helper
import labelreg.networks as network
import labelreg.apps as app
import labelreg.utils as util
import labelreg.losses as loss

def find_test_metrics(mov_img_dir,fix_img_dir,mov_lab_dir,fix_lab_dir,mod_dir):
    tf.reset_default_graph()

    # 1 - images to register
    reader_moving_image, reader_fixed_image, reader_moving_label, reader_fixed_label = helper.get_data_readers(mov_img_dir,
                                                                                                               fix_img_dir,
                                                                                                              mov_lab_dir,
                                                                                                              fix_lab_dir)

    # 2 - graph
    # network for predicting ddf only
    ph_moving_image = tf.placeholder(tf.float32, [reader_moving_image.num_data]+reader_moving_image.data_shape+[1])
    ph_fixed_image = tf.placeholder(tf.float32, [reader_fixed_image.num_data]+reader_fixed_image.data_shape+[1])

    reg_net = network.build_network(network_type='local',
                                    minibatch_size=reader_moving_image.num_data,
                                    image_moving=ph_moving_image,
                                    image_fixed=ph_fixed_image)

    ph_moving_label = tf.placeholder(tf.float32, [reader_moving_image.num_data]+reader_moving_image.data_shape+[1])
    ph_fixed_label = tf.placeholder(tf.float32, [reader_fixed_image.num_data]+reader_fixed_image.data_shape+[1])

    warped_moving_label = reg_net.warp_image(ph_moving_label)

    dice = util.compute_binary_dice(warped_moving_label, ph_fixed_label)
    dist = util.compute_centroid_distance(warped_moving_label, ph_fixed_label)
    # restore the trained weights
    saver = tf.train.Saver()
    sess = tf.Session()
    saver.restore(sess, mod_dir)

    test_labels = [1,12,18,19,23]
    dist_net = []
    for lidx in test_labels:
        testFeed = {ph_moving_image: reader_moving_image.get_data(),
                    ph_fixed_image: reader_fixed_image.get_data(),
                   ph_moving_label: reader_moving_label.get_data(label_indices=[lidx]),
                   ph_fixed_label: reader_fixed_label.get_data(label_indices=[lidx])}

        distance = sess.run(dist,feed_dict=testFeed)
        dist_net.append(distance)

    median_distance = np.median(np.sqrt(np.mean(np.vstack(dist_net)**2,axis=0)))

    testFeed = {ph_moving_image: reader_moving_image.get_data(),
                ph_fixed_image: reader_fixed_image.get_data(),
               ph_moving_label: reader_moving_label.get_data(label_indices=[1]),
               ph_fixed_label: reader_fixed_label.get_data(label_indices=[1])}

    median_dice = np.median(sess.run(dice,feed_dict=testFeed))
    return median_distance,median_dice




In [2]:
mov_img_dir = r'C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\test\moving_images'
fix_img_dir = r'C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\test\fixed_images'

# for test only
mov_lab_dir = r'C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\test\moving_labels'
fix_lab_dir = r'C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\test\fixed_labels'

mod_dir =  r'C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\saved_model2\model.ckpt'

med_dist,med_dice = find_test_metrics(mov_img_dir,fix_img_dir,mov_lab_dir,fix_lab_dir,mod_dir)

INFO:tensorflow:Restoring parameters from C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\saved_model2\model.ckpt


In [3]:
print('Target registration error: {}, Dice: {}'.format(med_dist,med_dice))

Target registration error: 1.8690412044525146, Dice: 0.5674880743026733


In [4]:
mod_dir =  r'C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\pruned_model2\pruned_model2.ckpt'

med_dist_pr,med_dice_pr = find_test_metrics(mov_img_dir,fix_img_dir,mov_lab_dir,fix_lab_dir,mod_dir)

print('After pruning Target registration error: {}, Dice: {}'.format(med_dist_pr,med_dice_pr))

INFO:tensorflow:Restoring parameters from C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\pruned_model2\pruned_model2.ckpt
After pruning Target registration error: 1.006929636001587, Dice: 0.6244825124740601


In [5]:
mod_dir =  r'C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\pruned_model_retrain2\model.ckpt'

med_dist_pr,med_dice_pr = find_test_metrics(mov_img_dir,fix_img_dir,mov_lab_dir,fix_lab_dir,mod_dir)

print('After pruning and retraining Target registration error: {}, Dice: {}'.format(med_dist_pr,med_dice_pr))

INFO:tensorflow:Restoring parameters from C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\pruned_model_retrain2\model.ckpt
After pruning and retraining Target registration error: 2.517721652984619, Dice: 0.5125890374183655


In [6]:
mod_dir =  r'C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\quantized_model2\model.ckpt'

med_dist_pr,med_dice_pr = find_test_metrics(mov_img_dir,fix_img_dir,mov_lab_dir,fix_lab_dir,mod_dir)

print('After quantization Target registration error: {}, Dice: {}'.format(med_dist_pr,med_dice_pr))

INFO:tensorflow:Restoring parameters from C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\quantized_model2\model.ckpt
After quantization Target registration error: 2.517721652984619, Dice: 0.5125890374183655


In [7]:
mod_dir =  r'C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\saved_model3\model.ckpt'

med_dist,med_dice = find_test_metrics(mov_img_dir,fix_img_dir,mov_lab_dir,fix_lab_dir,mod_dir)
print('Target registration error: {}, Dice: {}'.format(med_dist,med_dice))

INFO:tensorflow:Restoring parameters from C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\saved_model3\model.ckpt
Target registration error: 2.619112491607666, Dice: 0.4747452735900879


In [8]:
mod_dir =  r'C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\pruned_model3\pruned_model3.ckpt'

med_dist,med_dice = find_test_metrics(mov_img_dir,fix_img_dir,mov_lab_dir,fix_lab_dir,mod_dir)
print('After Pruning Target registration error: {}, Dice: {}'.format(med_dist,med_dice))

INFO:tensorflow:Restoring parameters from C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\pruned_model3\pruned_model3.ckpt
After Pruning Target registration error: 1.092018723487854, Dice: 0.6238847970962524


In [2]:
mov_img_dir = r'C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\test\moving_images'
fix_img_dir = r'C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\test\fixed_images'

# for test only
mov_lab_dir = r'C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\test\moving_labels'
fix_lab_dir = r'C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\test\fixed_labels'

mod_dir =  r'C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\saved_model4\model.ckpt'

med_dist,med_dice = find_test_metrics(mov_img_dir,fix_img_dir,mov_lab_dir,fix_lab_dir,mod_dir)
print('Target registration error: {}, Dice: {}'.format(med_dist,med_dice))

INFO:tensorflow:Restoring parameters from C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\saved_model4\model.ckpt
Target registration error: 2.440162420272827, Dice: 0.4699794352054596


In [3]:
mod_dir =  r'C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\pruned_model4\model.ckpt'

med_dist,med_dice = find_test_metrics(mov_img_dir,fix_img_dir,mov_lab_dir,fix_lab_dir,mod_dir)
print('After Pruning Target registration error: {}, Dice: {}'.format(med_dist,med_dice))

INFO:tensorflow:Restoring parameters from C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\pruned_model4\model.ckpt
After Pruning Target registration error: 1.205574631690979, Dice: 0.6047003269195557
